In [1]:
# David Ouyang 10/2/2019

# Notebook which iterates through a folder, including subfolders, 
# and convert DICOM files to AVI files of a defined size (natively 112 x 112)

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
#from scipy.misc import imread
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
from pydicom.pixel_data_handlers.util import convert_color_space


sys.path.append('/home/txl4827/projects/MESA')
from utils import *
from pathlib import Path

import pandas as pd
import os, sys, glob
print(sys.version)

destinationFolder = '/data/tom/MESA_Echos/alla4c'



3.10.4 (main, Mar 31 2022, 08:41:55) [GCC 7.5.0]


In [2]:
shahdir = '/mnt'
path ='/mnt'
proj ='MESA_Echos'
proj_export_path = os.path.join('/data/tom', proj, 'temp')
proj_work_path =  os.path.join('/home/txl4827', proj, 'temp')
path_to_dicom = Path('/data2/NMEcho/jtw_echo2/dicoms')
path_to_model = Path('/data2/NMEcho/echo_testing/models')

In [3]:
path_to_csv = '/data/tom/MESA_Echos/temp'
test_csv = 'results_5010446_mnt_MESA_Echos_Drive2_Volume3_GEMS_IMG_2017_MAY.csv'

output_path = '/data/tom/MESA_Echos/view_verify/'

In [4]:
paths = glob.glob(path_to_csv + '/*.csv')

In [5]:
output_pd = pd.DataFrame()
for csv in paths:
    temp_pd = pd.read_csv(csv)
    temp_pd = temp_pd[temp_pd.prob_a4c > 0.8] #Only spot checking a4c views
    output_pd = pd.concat([output_pd, temp_pd])
output_pd = output_pd.reset_index()

In [6]:
output_pd = output_pd.assign(out_path=output_pd.dicom_location.str.cat(output_pd.study.str.split('/').apply(lambda x: x[-1]), sep='/'))

In [7]:
outpaths = output_pd.out_path

In [8]:
# Dependencies you might need to run code
# Commonly missing

#!pip install pydicom
#!pip install opencv-python


In [9]:
def mask(output):
    dimension = output.shape[0]
    
    # Mask pixels outside of scanning sector
    m1, m2 = np.meshgrid(np.arange(dimension), np.arange(dimension))
    

    mask = ((m1+m2)>int(dimension/2) + int(dimension/10)) 
    mask *=  ((m1-m2)<int(dimension/2) + int(dimension/10))
    mask = np.reshape(mask, (dimension, dimension)).astype(np.int8)
    maskedImage = cv2.bitwise_and(output, output, mask = mask)
    
    #print(maskedImage.shape)
    
    return maskedImage



In [10]:
def makeVideo(fileToProcess, destinationFolder):
    try:
        fileName = fileToProcess.split('/')[-1] #\\ if windows, / if on mac or sherlock
                                            #hex(abs(hash(fileToProcess.split('/')[-1]))).upper()
        print(fileName)
        if not os.path.isdir(os.path.join(destinationFolder,fileName)):
            print(os.path.join(destinationFolder,fileName))
            dataset = dicom.dcmread(fileToProcess, force=True)
            testarray = dataset.pixel_array
            testarray = convert_color_space(testarray, 'YBR_FULL', 'RGB')
           
        
            frame0 = testarray[0]
            
            mean = np.mean(frame0, axis=1)
            mean = np.mean(mean, axis=1)
            
            yCrop = np.where(mean<1)[0][0] #what does this statement do
            testarray = testarray[:, yCrop:, :, :]
            print(3)
            bias = int(np.abs(testarray.shape[2] - testarray.shape[1])/2)
            if bias>0:
                if testarray.shape[1] < testarray.shape[2]:
                    testarray = testarray[:, :, bias:-bias, :]
                else:
                    testarray = testarray[:, bias:-bias, :, :]


            print(testarray.shape)
            frames,height,width,channels = testarray.shape

            fps = 30

            try:
                fps = dataset[(0x18, 0x40)].value
            except:
                print("couldn't find frame rate, default to 30")
            
            print('im at the writing part')
            fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
            video_filename = os.path.join(destinationFolder, fileName + '.avi')
            out = cv2.VideoWriter(video_filename, fourcc, fps, cropSize)

            print(4)
            for i in range(frames):

                outputA = testarray[i,:,:,0]
                smallOutput = outputA[int(height/10):(height - int(height/10)), int(height/10):(height - int(height/10))]

                # Resize image
                output = cv2.resize(smallOutput, cropSize, interpolation = cv2.INTER_CUBIC)

                finaloutput = mask(output)


                finaloutput = cv2.merge([finaloutput,finaloutput,finaloutput])
                out.write(finaloutput)

            out.release()

        else:
            print(fileName,"hasAlreadyBeenProcessed")
    except:
        print("something filed, not sure what, have to debug", fileName)
    return 0

In [11]:
AllA4cNames = None # = "Input Folder Name"
count = 0
    
cropSize = (112,112)

folder = outpaths #list of directories # = os.listdir(AllA4cNames)



In [ ]:
for directories in folder:
    count += 1
    VideoPath = os.path.join(directories)
    subcontent = '/'.join(directories.split('/')[-2:])
    print(count, directories)
    if not os.path.exists(os.path.join(destinationFolder, subcontent + ".avi")):
        makeVideo(VideoPath, destinationFolder)
    else:
        print("Already did this file", VideoPath)
    
# for folder in subfolders:
#     print(folder)
    
#     for content in os.listdir(os.path.join(AllA4cNames, folder)):
#         for subcontent in os.listdir(os.path.join(AllA4cNames, folder, content)):
#             count += 1
            

#             VideoPath = os.path.join(AllA4cNames, folder, content, subcontent)

#             print(count, folder, content, subcontent)

#             if not os.path.exists(os.path.join(destinationFolder,subcontent + ".avi")):
#                 makeVideo(VideoPath, destinationFolder)
#             else:
#                 print("Already did this file", VideoPath)


print(len(AllA4cFilenames))

1 /mnt/MESA_Echos/Drive2/Volume3/GEMS_IMG/2017_APR/18/GM160345/H4IGA282
H4IGA282
/data/tom/MESA_Echos/alla4c/H4IGA282
3
(58, 708, 708, 3)
im at the writing part
4
2 /mnt/MESA_Echos/Drive2/Volume3/GEMS_IMG/2017_APR/18/GM160345/H4IG9T02
H4IG9T02
/data/tom/MESA_Echos/alla4c/H4IG9T02
3
(179, 708, 708, 3)
im at the writing part
4
3 /mnt/MESA_Echos/Drive2/Volume3/GEMS_IMG/2017_APR/18/GM160345/H4IGH602
H4IGH602
/data/tom/MESA_Echos/alla4c/H4IGH602
3
(143, 708, 708, 3)
im at the writing part
4
4 /mnt/MESA_Echos/Drive2/Volume3/GEMS_IMG/2017_APR/18/GM160345/H4IGA2O2
H4IGA2O2
/data/tom/MESA_Echos/alla4c/H4IGA2O2
3
(172, 708, 708, 3)
im at the writing part
4
5 /mnt/MESA_Echos/Drive2/Volume3/GEMS_IMG/2017_APR/18/GM160345/H4IGH0O2
H4IGH0O2
/data/tom/MESA_Echos/alla4c/H4IGH0O2
3
(156, 708, 708, 3)
im at the writing part
4
6 /mnt/MESA_Echos/Drive2/Volume3/GEMS_IMG/2017_APR/18/GM160345/H4IG9SO4
H4IG9SO4
/data/tom/MESA_Echos/alla4c/H4IG9SO4
3
(62, 708, 708, 3)
im at the writing part
4
7 /mnt/MESA_Echos/

In [ ]:
filedirs = mesa_filedirs(os.path.join(path, proj))
filedirs

In [65]:
def list_paths2(files = filedirs, stills=True):
    exportlist = {}
    num_studies = 0
    for k, elem in files.items():
        temp = elem
        for pt, studies in temp.patients.items():
            exportlist[pt.patientid] = []
            num_studies += len(studies)
            for study in studies:
                if stills is False and study.frame == 1:
                    pass
                else:
                    export_study = [os.path.join(elem.path, '/'.join(study.path[:-1]))]
                    exportlist[pt.patientid] = export_study
    print(num_studies)
    return exportlist

In [12]:
folder[0]

'/mnt/MESA_Echos/Drive2/Volume3/GEMS_IMG/2017_APR/18/GM160345/H4IGA282'

In [48]:
a = dicom.dcmread(folder[0], force=True)

In [49]:
testarray = a.pixel_array

In [50]:
testarray.shape

(58, 708, 1016, 3)

In [51]:
testarray = convert_color_space(testarray, 'YBR_FULL', 'RGB')
frame0 = testarray[0]

In [52]:
mean = np.mean(frame0, axis=1)
mean = np.mean(mean, axis=1)

In [53]:
yCrop = np.where(mean<1)[0][0]
testarray = testarray[:, yCrop:, :, :]

In [54]:
bias = int(np.abs(testarray.shape[2] - testarray.shape[1])/2)
if bias>0:
    if testarray.shape[1] < testarray.shape[2]:
        print('here')
        testarray = testarray[:, :, bias:-bias, :]
    else:
        testarray = testarray[:, bias:-bias, :, :]


here


In [55]:
print(testarray.shape)
frames,height,width,channels = testarray.shape

fps = 30

try:
    fps = dataset[(0x18, 0x40)].value
except:
    print("couldn't find frame rate, default to 30")

(58, 708, 708, 3)
couldn't find frame rate, default to 30


In [ ]:
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
video_filename = os.path.join(destinationFolder, fileName + '.avi')
out = cv2.VideoWriter(video_filename, fourcc, fps, cropSize)

In [26]:
            


            



print(4)
for i in range(frames):

    outputA = testarray[i,:,:,0]
    smallOutput = outputA[int(height/10):(height - int(height/10)), int(height/10):(height - int(height/10))]

    # Resize image
    output = cv2.resize(smallOutput, cropSize, interpolation = cv2.INTER_CUBIC)

    finaloutput = mask(output)


    finaloutput = cv2.merge([finaloutput,finaloutput,finaloutput])
    out.write(finaloutput)

out.release()

array([[[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        ...,

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]],


       [[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0,

In [ ]:
testarray            